## 序言
前几天有朋友跟我说我前一阵写的太空了，说应该结合链家的数据写．

我说链家外网的数据太简单了，规模小，特征稀疏，无法达到令人满意的细粒度(文章结尾会进行总结)．

不过也不是不能用．

那我们这段时间就来分析分析链家及贝壳外网的租房数据，看能挖掘出多少有用的东西．

## 爬取数据
我们爬取了贝壳网９月１号上海市所有商圈的数据，得到了租房数据集，二手房数据集，及小区二手房均价数据集，代码就不贴了，非常简单．

## 载入数据集

In [1]:
import pandas as pd
import numpy as np
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
bkzf = pd.read_csv("bkzufang.csv")
subway = pd.read_csv("subway.csv")
xiaoqu = pd.read_csv("xiaoqu.csv")

In [2]:
subway.drop(['Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], axis=1, inplace=True) 
bkzf.drop(['Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'], axis=1, inplace=True) 
xiaoqu.drop(['Unnamed: 3'], axis=1, inplace=True) 

In [3]:
bkzf.head()

,xingzhengqu,shangqu,zulinfangshi,xiaoqu,huxing,area,price
0,宝山,大场镇,合租,招商中环华府,5居室,17,1500
1,宝山,大场镇,合租,招商中环华府,4居室,9,1530
2,宝山,大场镇,合租,智华苑,3居室,19,1780
3,宝山,大场镇,合租,乾皓苑,3居室,9,1360
4,宝山,大场镇,合租,乾皓苑,3居室,9,1460


In [4]:
subway.head()

,subway,xiaoqu,has_subway,subway_number,subway_station,distance
0,暂无数据,育秀五区,0,NaN,NaN,NaN
1,暂无数据,东方龙苑,0,NaN,NaN,NaN
2,距地铁5号线华宁路237米,红旗小区,1,5,华宁路,237
3,暂无数据,中科大学村,0,NaN,NaN,NaN
4,暂无数据,金球怡云花园,0,NaN,NaN,NaN


In [5]:
xiaoqu.head()

,xiaoqu,xiaoqujunjia,zaishou
0,乾溪新村,40757元/m2,20套在售二手房
1,上大阳光乾和园,41471元/m2,16套在售二手房
2,同济城市阳光,51888元/m2,7套在售二手房
3,骏华苑,44267元/m2,4套在售二手房
4,南大路180弄,38121元/m2,10套在售二手房


## 重复数据删除

In [7]:
subway.drop_duplicates(subset=['xiaoqu'], keep='first', inplace=True)
xiaoqu.drop_duplicates(subset=['xiaoqu'], keep='first', inplace=True)

## 按照小区信息拼接表格

In [8]:
data = pd.merge(bkzf, subway, how='left', on=['xiaoqu'])
data = pd.merge(data, xiaoqu, how='left', on=['xiaoqu'])

## 调用百度地图api进行地理编码

我们得到了房源所在商圈小区的信息，通过这些信息我们可以得知房源的位置，但是仅靠这种文字信息并不能简单直白的概括不同房源的相对位置的差别，我们可以利用百度地图的api将其转化为具体的经纬度进行利用，获取房源所在小区的具体坐标，为后面有效的定量分析做铺垫．

In [9]:
data['location'] = data['xingzhengqu'] + [' '] +  data['shangqu'] + [' '] + data['xiaoqu']

In [10]:
import urllib.parse
import hashlib
import requests
import json

In [11]:
# 为 dataframe 添加多列数据
def add_columns(df, data, new_columns):
    # dataframe 的列名
    columns = list(df.columns)
    # 列名新增列
    columns.extend(new_columns)
    # 重新生成一个新的 dataframe 并返回
    df = pd.DataFrame(np.concatenate(
        (df.values, data), axis=1), columns=columns)
    return df

In [12]:
def getLonAndLat(location):

    ak = 'NeS0f3snB0G4xvqjrXkd6ZHUxLfCRU7t'
    sk = 'thfZVoWwjfga9O8qfTTH4jGtpxDButL2'
    
    queryStr = '/geocoding/v3/?address=上海 {}&output=json&ak={}'.format(
        location, ak)
    
 # 对queryStr进行转码，safe内的保留字符不转换
    encodedStr = urllib.parse.quote(queryStr, safe="/:=&?#+!$,;'@()*[]")

    # 追加sk
    rawStr = encodedStr + sk

    # 生成 url 并进行 md5 加密
    queryStr = 'http://api.map.baidu.com{}&sn={}'.format(
        queryStr, hashlib.md5(urllib.parse.quote_plus(rawStr).encode()).hexdigest())
    r = requests.get(queryStr)
    r = json.loads(r.content.decode())
    # 获取经纬度
    location = r['result']['location']
    return location['lng'], location['lat']


getLonAndLat('松江 松江大学城 万达广场')  # 测试用例

(121.2513813796999, 31.064262980922994)

可以看到，通过我们定义的函数，可成功得到某个地点的经纬度数据．

需要注意的是，由于国家保密政策，我们通过这种方式获取到的经纬度数据都是存在偏移的，后续进行可视化的时候我们需要进行转换或者人为的修正．

通过百度地图开发者平台提供的ak和sk，我们可以很方便的为每个房源进行地理编码从而得到经纬度信息．

In [24]:
from tqdm import tqdm_notebook
locations = set(data['location'].values.tolist())  # 去除重复地址
location_data_dict = {}

for loc in tqdm_notebook(locations):
    # 获取地址经纬度
    lon, lat = getLonAndLat(loc)
    location_data_dict[loc] = [lon, lat]

# 添加到 Dataframe
location_data = [location_data_dict[loc] for loc in data['location']]
data = add_columns(data, location_data, ['lon', 'lat'])
data.head()

,xingzhengqu,shangqu,zulinfangshi,xiaoqu,huxing,area,price,subway,has_subway,subway_number,subway_station,distance,xiaoqujunjia,zaishou,location,lon,lat
0,宝山,大场镇,合租,招商中环华府,5居室,17平米,1500,NaN,NaN,NaN,NaN,NaN,64820元/m2,4套在售二手房,宝山 大场镇 招商中环华府,121.397,31.3102
1,宝山,大场镇,合租,招商中环华府,4居室,9平米,1530,NaN,NaN,NaN,NaN,NaN,64820元/m2,4套在售二手房,宝山 大场镇 招商中环华府,121.397,31.3102
2,宝山,大场镇,合租,智华苑,3居室,19平米,1780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,宝山 大场镇 智华苑,121.42,31.3125
3,宝山,大场镇,合租,乾皓苑,3居室,9平米,1360,NaN,NaN,NaN,NaN,NaN,暂无,0套在售二手房,宝山 大场镇 乾皓苑,121.401,31.3119
4,宝山,大场镇,合租,乾皓苑,3居室,9平米,1460,NaN,NaN,NaN,NaN,NaN,暂无,0套在售二手房,宝山 大场镇 乾皓苑,121.401,31.3119


In [14]:
data.head(15)

,Unnamed: 0,xingzhengqu,shangqu,zulinfangshi,xiaoqu,huxing,area,price,subway,has_subway,subway_number,subway_station,distance,xiaoqujunjia,zaishou,location,lon,lat
0,0,宝山,大场镇,合租,招商中环华府,5居室,17,1500,NaN,NaN,NaN,NaN,NaN,64820元/m2,4套在售二手房,宝山 大场镇 招商中环华府,121.396598,31.310192
1,1,宝山,大场镇,合租,招商中环华府,4居室,9,1530,NaN,NaN,NaN,NaN,NaN,64820元/m2,4套在售二手房,宝山 大场镇 招商中环华府,121.396598,31.310192
2,2,宝山,大场镇,合租,智华苑,3居室,19,1780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,宝山 大场镇 智华苑,121.420300,31.312524
3,3,宝山,大场镇,合租,乾皓苑,3居室,9,1360,NaN,NaN,NaN,NaN,NaN,暂无,0套在售二手房,宝山 大场镇 乾皓苑,121.400931,31.311855
4,4,宝山,大场镇,合租,乾皓苑,3居室,9,1460,NaN,NaN,NaN,NaN,NaN,暂无,0套在售二手房,宝山 大场镇 乾皓苑,121.400931,31.311855
5,5,宝山,大场镇,合租,智华苑,3居室,20,2100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,宝山 大场镇 智华苑,121.420300,31.312524
6,6,宝山,大场镇,合租,乾皓苑,3居室,30,2000,NaN,NaN,NaN,NaN,NaN,暂无,0套在售二手房,宝山 大场镇 乾皓苑,121.400931,31.311855
7,7,宝山,大场镇,合租,智华苑,3居室,20,2100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,宝山 大场镇 智华苑,121.420300,31.312524
8,8,宝山,大场镇,合租,智华苑,3居室,20,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,宝山 大场镇 智华苑,121.420300,31.312524
9,9,宝山,大场镇,合租,智华苑,3居室,20,2100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,宝山 大场镇 智华苑,121.420300,31.312524


In [27]:
data.drop(['Unnamed: 0'], axis=1, inplace=True) 

## 处理缺失值

In [15]:
def null_count(data):  # 定义 null 值查找函数，函数名 null_count
    null_data = data.isnull().sum()  # 查找各个特征 null 值并计算数量
    null_data = null_data.drop(null_data[null_data == 0].index).sort_values(
        ascending=False)  # 删除数目为零的特征，降序排列
    return null_data  # 返回结果

null_count(data)  # 调用 null_count 函数统计 data 的 null，输出结果

distance          34281
subway_station    34281
subway_number     34281
has_subway        21119
subway            21119
zaishou            4987
xiaoqujunjia       4987
location              1
dtype: int64

In [17]:
data['xiaoqujunjia'].fillna('暂无',inplace=True) 

In [18]:
data['zaishou'].fillna('0套在售二手房',inplace=True) 

## 从文字信息中提取数据特征

In [ ]:
xiaoqujunjia_info = data['xiaoqujunjia'].value_counts()
count_xiaoqujunjia = pd.DataFrame({"xiaoqujunjia_info" : xiaoqujunjia_info.index, "ValueCount":xiaoqujunjia_info})
count_xiaoqujunjia

In [20]:
data['xiaoqujunjia'].replace('暂无', '未知元/m2', inplace=True) 

In [21]:
import re
pattern = r'(.*)元/m2'
junjia = []
for xiaoqujunjia in data['xiaoqujunjia']:
    junjia.append(list(re.findall(pattern, xiaoqujunjia)[:]))
# 添加新列
data.drop(['xiaoqujunjia'], axis=1, inplace=True) 
data = add_columns(data, junjia, ['xiaoqujunjia'])
data.head()

,Unnamed: 0,xingzhengqu,shangqu,zulinfangshi,xiaoqu,huxing,area,price,subway,has_subway,subway_number,subway_station,distance,zaishou,location,lon,lat,xiaoqujunjia
0,0,宝山,大场镇,合租,招商中环华府,5居室,17,1500,NaN,NaN,NaN,NaN,NaN,4套在售二手房,宝山 大场镇 招商中环华府,121.397,31.3102,64820
1,1,宝山,大场镇,合租,招商中环华府,4居室,9,1530,NaN,NaN,NaN,NaN,NaN,4套在售二手房,宝山 大场镇 招商中环华府,121.397,31.3102,64820
2,2,宝山,大场镇,合租,智华苑,3居室,19,1780,NaN,NaN,NaN,NaN,NaN,0套在售二手房,宝山 大场镇 智华苑,121.42,31.3125,未知
3,3,宝山,大场镇,合租,乾皓苑,3居室,9,1360,NaN,NaN,NaN,NaN,NaN,0套在售二手房,宝山 大场镇 乾皓苑,121.401,31.3119,未知
4,4,宝山,大场镇,合租,乾皓苑,3居室,9,1460,NaN,NaN,NaN,NaN,NaN,0套在售二手房,宝山 大场镇 乾皓苑,121.401,31.3119,未知


In [ ]:
sale_info = data['zaishou'].value_counts()
count_sale = pd.DataFrame({"sale" : sale_info.index, "ValueCount":sale_info})
count_sale

In [23]:
pattern = r'(.*)套在售二手房'
sale_count = []
for zaishou in data['zaishou']:
    sale_count.append(list(re.findall(pattern, zaishou)[:]))
# 添加新列
data.drop(['zaishou'], axis=1, inplace=True) 
data = add_columns(data, sale_count, ['sale_count'])
data.head(10)

,Unnamed: 0,xingzhengqu,shangqu,zulinfangshi,xiaoqu,huxing,area,price,subway,has_subway,subway_number,subway_station,distance,location,lon,lat,xiaoqujunjia,sale_count
0,0,宝山,大场镇,合租,招商中环华府,5居室,17,1500,NaN,NaN,NaN,NaN,NaN,宝山 大场镇 招商中环华府,121.397,31.3102,64820,4
1,1,宝山,大场镇,合租,招商中环华府,4居室,9,1530,NaN,NaN,NaN,NaN,NaN,宝山 大场镇 招商中环华府,121.397,31.3102,64820,4
2,2,宝山,大场镇,合租,智华苑,3居室,19,1780,NaN,NaN,NaN,NaN,NaN,宝山 大场镇 智华苑,121.42,31.3125,未知,0
3,3,宝山,大场镇,合租,乾皓苑,3居室,9,1360,NaN,NaN,NaN,NaN,NaN,宝山 大场镇 乾皓苑,121.401,31.3119,未知,0
4,4,宝山,大场镇,合租,乾皓苑,3居室,9,1460,NaN,NaN,NaN,NaN,NaN,宝山 大场镇 乾皓苑,121.401,31.3119,未知,0
5,5,宝山,大场镇,合租,智华苑,3居室,20,2100,NaN,NaN,NaN,NaN,NaN,宝山 大场镇 智华苑,121.42,31.3125,未知,0
6,6,宝山,大场镇,合租,乾皓苑,3居室,30,2000,NaN,NaN,NaN,NaN,NaN,宝山 大场镇 乾皓苑,121.401,31.3119,未知,0
7,7,宝山,大场镇,合租,智华苑,3居室,20,2100,NaN,NaN,NaN,NaN,NaN,宝山 大场镇 智华苑,121.42,31.3125,未知,0
8,8,宝山,大场镇,合租,智华苑,3居室,20,1900,NaN,NaN,NaN,NaN,NaN,宝山 大场镇 智华苑,121.42,31.3125,未知,0
9,9,宝山,大场镇,合租,智华苑,3居室,20,2100,NaN,NaN,NaN,NaN,NaN,宝山 大场镇 智华苑,121.42,31.3125,未知,0


In [24]:
data.tail(10)

,Unnamed: 0,xingzhengqu,shangqu,zulinfangshi,xiaoqu,huxing,area,price,subway,has_subway,subway_number,subway_station,distance,location,lon,lat,xiaoqujunjia,sale_count
49859,49859,闸北,闸北公园,整租,海珀星晖,3室2厅,90,13000,NaN,NaN,NaN,NaN,NaN,闸北 闸北公园 海珀星晖,121.463,31.277,93000,4
49860,49860,闸北,闸北公园,整租,和源大楼,1室1厅,57,60000,NaN,NaN,NaN,NaN,NaN,闸北 闸北公园 和源大楼,121.474,31.271,57214,5
49861,49861,闸北,闸北公园,整租,和源大楼,1室1厅,144,60000,NaN,NaN,NaN,NaN,NaN,闸北 闸北公园 和源大楼,121.474,31.271,57214,5
49862,49862,闸北,闸北公园,整租,和源大楼,1室0厅,67,60000,NaN,NaN,NaN,NaN,NaN,闸北 闸北公园 和源大楼,121.474,31.271,57214,5
49863,49863,闸北,闸北公园,整租,和源大楼,1室1厅,129,60000,NaN,NaN,NaN,NaN,NaN,闸北 闸北公园 和源大楼,121.474,31.271,57214,5
49864,49864,闸北,闸北公园,整租,和源大楼,1室1厅,137,60000,NaN,NaN,NaN,NaN,NaN,闸北 闸北公园 和源大楼,121.474,31.271,57214,5
49865,49865,闸北,闸北公园,整租,嘉利明珠城,2室1厅,86,8200,距地铁1号线延长路998米,1,1,延长路,998,闸北 闸北公园 嘉利明珠城,121.47,31.2794,76237,18
49866,49866,闸北,闸北公园,整租,中山北路255弄,2室1厅,53,8200,距地铁8号线西藏北路261米,1,8,西藏北路,261,闸北 闸北公园 中山北路255弄,121.475,31.2704,57701,8
49867,49867,闸北,闸北公园,整租,申地苑,2室2厅,96,9000,NaN,NaN,NaN,NaN,NaN,闸北 闸北公园 申地苑,121.473,31.2715,62204,3
49868,49868,闸北,闸北公园,整租,天祥华府,2室2厅,90,14000,距地铁8号线西藏北路553米,1,8,西藏北路,553,闸北 闸北公园 天祥华府,121.473,31.2745,79989,3


## 预处理后的数据说明

- 数值特征  

|变量名| 变量描述 |  
| -- | -- |
|**房屋客观数据**|　|　|
|area|房屋面积| 
|**配套环境数据**|　|　|
|lon|经度| 
|lat|纬度| 
|distance|距地铁站距离| 
|**房屋价值数据**|　|　|
|price|一个月租金| 
|**小区行情数据**|　|　|
|sale_count|小区在售房源数量| 
|xiaoqujunjia|小区楼盘每平米售价| 


- 类别特征    

|变量名| 变量描述 |  
| -- | -- |
|**地理位置属性**|　|　|
|xingzhengqu|行政区信息| 
|shangqu|商区信息| 
|xiaoqu|小区名称| 
|has_subway|附近有无地铁| 
|subway_number|附近地铁线路| 
|subway_station|附近地铁站| 
|**房屋客观属性**|　|　|
|huxing|房屋户型| 
|zulinfangshi|租赁方式| 

## 链家贝壳数据分析存在的问题

- 数据规模小，楼盘房源密度低

数据规模和楼盘房源密度很大程度上是由市场因素决定的，在租在售房源通常只占一个楼盘很小的比重，这是我们分析问题时面临的最大的问题，数据本身限制了数据建模的解释能力

- 特征对决策影响不大

我们获取的特征都是房源的客观属性以及业主的主观定价，这些都是业主方的单边信息，现实中决定房产交易的往往是业主客户中介的心态以及各种看不见的手，换句话说，促成交易的往往是时机而非产品本身

- 特征稀疏

特征非常稀疏，利用经验建模的模型收益很低